In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Tools

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def getLoader(path, samples, batch_size, shuff=False):
    load = []
    for f in samples:
        data = unpickle(path + f)
        x_set = data[b'data']
        t_set = np.array(data[b'labels'])

        # Shuffle
        if shuff:
            index = np.arange(x_set.shape[0])
            np.random.shuffle(index)
            x_set = x_set[index]
            t_set = t_set[index]

        x_set = x_set.reshape(x_set.shape[0], 3, 32, 32)
        x_set = x_set.transpose(0, 2, 3, 1).astype('uint8')

        for i in range(x_set.shape[0] // batch_size):
            load.append({'imgs': x_set[batch_size * i:batch_size * (i + 1)],
                         'targets': t_set[batch_size * i:batch_size * (i + 1)]})
    return load

# Layers

In [3]:
class Param:
    def __init__(self, value):
        self.value = value
        self.grad = np.zeros_like(value)

In [4]:
class ReLU:
    def __init__(self):
        self.deriv = None
    
    def __str__(self):
         return "ReLU()"
    
    def params(self):
        return {}
    
    def zero_grad(self):
        pass
    
    def forward(self, X, training):
        self.deriv = (X > 0)
        return X * self.deriv
    
    def backward(self, d_out):
        d_result = d_out * self.deriv
        return d_result

class Sigmoid:
    def __init__(self):
        self.deriv = None
    
    def __str__(self):
         return "Sigmoid()"

    def params(self):
        return {}
    
    def zero_grad(self):
        pass
    
    def forward(self, X, training):
        self.deriv = (1 / (1 + np.exp(-X))) * (1 - 1 / (1 + np.exp(-X)))
        return 1 / (1 + np.exp(-X))
    
    def backward(self, d_out):
        d_result = d_out * self.deriv
        return d_result

class Softmax:
    def __init__(self):
        self.deriv = None
    
    def __str__(self):
         return "Softmax()"

    def params(self):
        return {}
    
    def zero_grad(self):
        pass
    
    def forward(self, Z_last, training):
        Z_last -= np.max(Z_last, axis=1).T.reshape((Z_last.shape[0], 1))
        return np.exp(Z_last) / np.sum(np.exp(Z_last), axis=1).T.reshape((Z_last.shape[0], 1))
    
    def backward(self, d_out):
        return d_out

# Fully-Connected

In [5]:
class FullyConnected:
    def __init__(self, n_input, n_output):
        self.n_size = (n_input, n_output)
        self.W = Param(np.random.randn(n_output, n_input) * 0.1)
        self.B = Param(np.random.randn(1, n_output) * 0.1)

        self.Z_before = None
    
    def __str__(self):
         return "FullyConnected(n_input={}, n_output={})".format(*self.n_size)
    
    def params(self):
        return {'weight': self.W.value, 'bias': self.B.value}
    
    def zero_grad(self):
        self.W.grad = np.zeros_like(self.W.value)
        self.B.grad = np.zeros_like(self.B.value)

    def forward(self, X, training):
        self.Z_before = X.copy()
        return X @ self.W.value.T + self.B.value
    
    def backward(self, d_out):
        n = self.Z_before.shape[0]
        self.W.grad += (d_out.T @ self.Z_before) / n
        self.B.grad += np.sum(d_out, axis=0, keepdims=True) / n
        return d_out @ self.W.value

# CNN

https://github.com/SkalskiP/ILearnDeepLearning.py/blob/ba0b5ba589d4e656141995e8d1a06d44db6ce58d/01_mysteries_of_neural_networks/06_numpy_convolutional_neural_net/src/layers/convolutional.py#L187

## Tools

In [6]:
def get_im2col_idx(array_shape, filter_dim=(3, 3), pad=0, stride=1):
    n, c, h_in, w_in = array_shape
    h_f, w_f = filter_dim

    h_out = (h_in + 2 * pad - h_f) // stride + 1
    w_out = (w_in + 2 * pad - w_f) // stride + 1

    i0 = np.repeat(np.arange(h_f), w_f)
    i0 = np.tile(i0, c)
    i1 = stride * np.repeat(np.arange(h_out), w_out)
    j0 = np.tile(np.arange(w_f), h_f * c)
    j1 = stride * np.tile(np.arange(w_out), h_out)
    i = i0.reshape(-1, 1) + i1.reshape(1, -1)
    j = j0.reshape(-1, 1) + j1.reshape(1, -1)
    k = np.repeat(np.arange(c), h_f * w_f).reshape(-1, 1)
    return k, i, j


def im2col(array, filter_dim=(3, 3), pad=0, stride=1):
    _, c, _, _ = array.shape
    h_f, w_f = filter_dim
    array_pad = np.pad(
        array=array,
        pad_width=((0, 0), (0, 0), (pad, pad), (pad, pad)),
        mode='constant'
    )
    k, i, j = get_im2col_idx(
        array_shape=array.shape,
        filter_dim=filter_dim,
        pad=pad,
        stride=stride
    )
    cols = array_pad[:, k, i, j]
    return cols.transpose(1, 2, 0).reshape(h_f * w_f * c, -1)


def col2im(cols, array_shape, filter_dim=(3, 3), pad=0, stride=1):
    n, c, h_in, w_in = array_shape
    h_f, w_f = filter_dim
    h_pad, w_pad = h_in + 2 * pad, w_in + 2 * pad
    array_pad = np.zeros((n, c, h_pad, w_pad), dtype=cols.dtype)
    k, i, j = get_im2col_idx(
        array_shape=array_shape,
        filter_dim=filter_dim,
        pad=pad,
        stride=stride
    )
    cols_reshaped = cols.reshape(c * h_f * w_f, -1, n)
    cols_reshaped = cols_reshaped.transpose(2, 0, 1)
    np.add.at(array_pad, (slice(None), k, i, j), cols_reshaped)
    return array_pad[:, :, pad:pad+h_in, pad:pad+w_in]

## Layer

In [7]:
class Conv2D:
    def __init__(self, in_channels, out_channels, kernel_size=(3, 3), stride=1, padding='valid'):
        self.filter_size = kernel_size
        self.n_size = (in_channels, out_channels)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.W = Param(
            np.random.randn(kernel_size[0], kernel_size[1],
                            in_channels, out_channels) * 0.1
        )

        self.B = Param(np.random.randn(out_channels) * 0.1)

        self._padding = padding
        self._stride = stride
        self._Z_before = None
        self._cols = None

    def __str__(self):
         return "Conv2D(in_channels={}, out_channels={}, kernel_size=({}, {}), stride={}, padding={})"\
         .format(*self.n_size, *self.n_size, self._stride, self._padding)

    def params(self):
        return { 'W': self.W.value, 'B': self.B.value }
    
    def zero_grad(self):
        self.W.grad = np.zeros_like(self.W.value)
        self.B.grad = np.zeros_like(self.B.value)

    def __calculate_output_dims(self, input_dims):
        n, h_in, w_in, _ = input_dims
        h_f, w_f, _, n_f = self.W.value.shape
        if self._padding == 'same':
            return n, h_in, w_in, n_f
        elif self._padding == 'valid':
            h_out = (h_in - h_f) // self._stride + 1
            w_out = (w_in - w_f) // self._stride + 1
            return n, h_out, w_out, n_f
        else:
            raise Exception("Invalid padding!")

    def __calculate_pad_dims(self):
        if self._padding == 'same': # ZeroPadding
            h_f, w_f, _, _ = self.W.value.shape
            return (h_f - 1) // 2, (w_f - 1) // 2
        elif self._padding == 'valid':
            return 0, 0
        else:
            raise Exception("Invalid padding!")

    def forward(self, X, training):
        self._Z_before = np.copy(X)
        
        n, h_out, w_out, _ = self.__calculate_output_dims(input_dims=X.shape)
        h_f, w_f, _, n_f = self.W.value.shape
        pad = self.__calculate_pad_dims()
        w = np.transpose(self.W.value, (3, 2, 0, 1))

        self._cols = im2col(
            array=np.moveaxis(X, -1, 1),
            filter_dim=(h_f, w_f),
            pad=pad[0],
            stride=self._stride
        )

        result = w.reshape((n_f, -1)).dot(self._cols)
        output = result.reshape(n_f, h_out, w_out, n)

        return output.transpose(3, 1, 2, 0) + self.B.value


    def backward(self, d_out):
        n, h_out, w_out, _ = self.__calculate_output_dims(
            input_dims=self._Z_before.shape)
        h_f, w_f, _, n_f = self.W.value.shape
        pad = self.__calculate_pad_dims()

        self.B.grad += d_out.sum(axis=(0, 1, 2)) / n
        d_out_reshaped = d_out.transpose(3, 1, 2, 0).reshape(n_f, -1)

        w = np.transpose(self.W.value, (3, 2, 0, 1))
        dw = d_out_reshaped.dot(self._cols.T).reshape(w.shape)
        self.W.grad += np.transpose(dw, (2, 3, 1, 0))

        output_cols = w.reshape(n_f, -1).T.dot(d_out_reshaped)

        output = col2im(
            cols=output_cols,
            array_shape=np.moveaxis(self._Z_before, -1, 1).shape,
            filter_dim=(h_f, w_f),
            pad=pad[0],
            stride=self._stride
        )
        return np.transpose(output, (0, 2, 3, 1))

# MaxPooling

In [8]:
class MaxPooling:
    def __init__(self, pool_size, stride):
        self._pool_size = pool_size
        self._stride = stride
        self._Z_before = None
        self._cache = {}

    def __str__(self):
         return "MaxPooling(pool_size=({}, {}), stride={})"\
         .format(*self._pool_size, self._stride)

    def params(self):
        return {}
    
    def zero_grad(self):
        pass

    def _save_mask(self, X, cords):
        mask = np.zeros_like(X)
        n, h, w, c = X.shape
        X = X.reshape(n, h * w, c)
        idx = np.argmax(X, axis=1)

        n_idx, c_idx = np.indices((n, c))
        mask.reshape(n, h * w, c)[n_idx, idx, c_idx] = 1
        self._cache[cords] = mask

    def forward(self, X, training):
        self._Z_before = np.copy(X)
        n, h_in, w_in, c = X.shape
        h_pool, w_pool = self._pool_size
        h_out = 1 + (h_in - h_pool) // self._stride
        w_out = 1 + (w_in - w_pool) // self._stride
        output = np.zeros((n, h_out, w_out, c))

        for i in range(h_out):
            for j in range(w_out):
                h_start = i * self._stride
                h_end = h_start + h_pool
                w_start = j * self._stride
                w_end = w_start + w_pool
                X_slice = X[:, h_start:h_end, w_start:w_end, :]
                self._save_mask(X_slice, (i, j))
                output[:, i, j, :] = np.max(X_slice, axis=(1, 2))
        return output

    def backward(self, d_out):
        output = np.zeros_like(self._Z_before)
        _, h_out, w_out, _ = d_out.shape
        h_pool, w_pool = self._pool_size

        for i in range(h_out):
            for j in range(w_out):
                h_start = i * self._stride
                h_end = h_start + h_pool
                w_start = j * self._stride
                w_end = w_start + w_pool
                output[:, h_start:h_end, w_start:w_end, :] += \
                    d_out[:, i:i + 1, j:j + 1, :] * self._cache[(i, j)]
        return output

    def params(self):
        return {}

# Flattener

In [9]:
class Flattener:
    def __init__(self):
        self.X_shape = None

    def __str__(self):
         return "Flattener()"

    def params(self):
        return {}
    
    def zero_grad(self):
        pass

    def forward(self, X, training):
        batch_size, height, width, channels = X.shape
        self.X_shape = X.shape
        return np.ravel(X).reshape(X.shape[0], -1)

    def backward(self, d_out):
        return d_out.reshape(self.X_shape)

    def params(self):
        return {}

# Dropout

In [10]:
class Dropout:
    def __init__(self, keep_prob):
        self._keep_prob = keep_prob
        self._mask = None
    
    def __str__(self):
         return "Dropout(keep_prob={})".format(self._keep_prob)

    def params(self):
        return {}
    
    def zero_grad(self):
        pass

    def forward(self, X, training):
        if training:
            self._mask = (np.random.rand(*X.shape) < self._keep_prob)
            return self._apply_mask(X, self._mask)
        else:
            return X

    def backward(self, d_out):
        return self._apply_mask(d_out, self._mask)

    def _apply_mask(self, array, mask):
        array *= mask
        array /= self._keep_prob
        return array

# Net

In [17]:
class NeuralNetwork:
    def __init__(self, layers):
        self.__layers = layers
        self.__optimizer = None
    
    def __getitem__(self, id):
        return self.__layers[id]

    def __str__(self):
        str_net = "NeuralNetwork {\n"
        for l in self.__layers:
            str_net += "\t{}\n".format(str(l))
        str_net += "}"
        return str_net

    def save_weights(self, name='weights'):
        f = open(r'{}.pickle'.format(name), 'wb')
        obj = self.params()
        pickle.dump(obj, f)
        print('The model is saved to file - {}.pickle!'.format(name))
        f.close()
    
    def load_weights(self, weights=None, path='weights.pickle'):
        if weights is None:
            f = open(path, 'rb')
            obj = pickle.load(f)
            f.close()
        else:
            obj = weights
        i = 0
        for l in self.__layers:
            if not hasattr(l, 'W'):
                continue
            l.W.value = obj[i]['weight']
            l.B.value = obj[i]['bias']
            i += 1

    def params(self):
        return [l.params() for l in self.__layers if hasattr(l, 'W')]
    
    def initNorm(self, sigma):
        for l in self.__layers:
            if hasattr(l, 'W'):
                l.W.value = np.random.randn(*l.W.value.shape) * sigma
                l.B.value = np.random.randn(*l.B.value.shape) * sigma
    
    def initHe(self):
        for l in self.__layers:
            if hasattr(l, 'W'):
                l.W.value = np.random.randn(*l.W.value.shape) * np.sqrt(2 / l.n_size[0])
                l.B.value = np.random.randn(*l.B.value.shape) * np.sqrt(2 / l.n_size[0])
    
    def initXavier(self):
        for l in self.__layers:
            if hasattr(l, 'W'):
                l.W.value = np.random.randn(*l.W.value.shape) * np.sqrt(2 / (l.n_size[0] + l.n_size[1]))
                l.B.value = np.random.randn(*l.B.value.shape) * np.sqrt(2 / (l.n_size[0] + l.n_size[1]))
    
    def zero_grad(self):
        for l in self.__layers:
            l.zero_grad()
    
    def set_optimizer(self, optimizer):
        self.__optimizer = optimizer
    
    def forward(self, x_set, training):
        x_ = np.copy(x_set)
        for l in self.__layers:
            x_ = l.forward(x_, training)
        return x_
    
    def backward(self, net_out, t_set):
        d_out = net_out.copy()
        t_mask = np.zeros(d_out.shape)
        t_mask[np.arange(len(t_set)), t_set.reshape(1, -1)] = 1
        d_out -= t_mask
        for l in reversed(self.__layers):
            d_out = l.backward(d_out)
    
    def step(self):
        self.__optimizer.update_weight(self.__layers)

# Optimizers

In [48]:
class Adam:
    def __init__(self, lr, reg=0, beta1=0.9, beta2=0.999, eps=1e-8):
        self._lr = lr
        self._reg = reg
        self._beta1 = beta1
        self._beta2 = beta2
        self._cache_v = {}
        self._cache_s = {}
        self._eps = eps
    
    def _init_cache(self, layers):
        for idx, l in enumerate(layers):
            if hasattr(l, 'W'):
                dw, db = l.W.grad, l.B.grad
                if dw is None or db is None:
                    continue

                dw_key, db_key = Adam._get_keys(idx)

                self._cache_v[dw_key] = np.zeros_like(dw)
                self._cache_v[db_key] = np.zeros_like(db)
                self._cache_s[dw_key] = np.zeros_like(dw)
                self._cache_s[db_key] = np.zeros_like(db)
        
    @staticmethod
    def _get_keys(idx):
        return f"dw{idx}", f"db{idx}"

    def update_weight(self, layers):
        if len(self._cache_s) == 0 or len(self._cache_v) == 0:
            self._init_cache(layers)

        if self._reg:
            for l in reversed(layers):
                if hasattr(l, 'W'):
                    l.W.grad += reg * 2 * l.W.value

        for idx, l in enumerate(layers):
            if hasattr(l, 'W'):
                dw_key, db_key = Adam._get_keys(idx)

                self._cache_v[dw_key] = self._beta1 * self._cache_v[dw_key] + \
                (1 - self._beta1) * l.W.grad
                self._cache_v[db_key] = self._beta1 * self._cache_v[db_key] + \
                    (1 - self._beta1) * l.B.grad

                self._cache_s[dw_key] = self._beta2 * self._cache_s[dw_key] + \
                    (1 - self._beta2) * np.square(l.W.grad)
                self._cache_s[db_key] = self._beta2 * self._cache_s[db_key] + \
                    (1 - self._beta2) * np.square(l.B.grad)

                dw = self._cache_v[dw_key] / (np.sqrt(self._cache_s[dw_key]) + self._eps)
                db = self._cache_v[db_key] / (np.sqrt(self._cache_s[db_key]) + self._eps)

                l.W.value -= self._lr * dw
                l.B.value -= self._lr * db

class SGD:
    def __init__(self, lr, reg=0, momentum=0.9):
        self._lr = lr
        self._reg = reg
        self._momentum = momentum
        self._cache_vx = {}
    
    def _init_cache(self, layers):
        for idx, l in enumerate(layers):
            if hasattr(l, 'W'):
                dw, db = l.W.grad, l.B.grad
                if dw is None or db is None:
                    continue

                dw_key, db_key = SGD._get_keys(idx)

                self._cache_vx[dw_key] = np.zeros_like(dw)
                self._cache_vx[db_key] = np.zeros_like(db)

    @staticmethod
    def _get_keys(idx):
        return f"dw{idx}", f"db{idx}"
    
    def update_weight(self, layers):
        if self._reg:
            for l in reversed(layers):
                if hasattr(l, 'W'):
                    l.W.grad += reg * 2 * l.W.value

        for idx, l in enumerate(layers):
            if hasattr(l, 'W'): 
                dw_key, db_key = SGD._get_keys(idx)

                self._cache_vx[dw_key] = self._momentum * self._cache_vx[dw_key] + l.W.grad
                self._cache_vx[db_key] = self._momentum * self._cache_vx[db_key] + l.B.grad

                l.W.value -= self._lr * self._cache_vx[dw_key]
                l.B.value -= self._lr * self._cache_vx[db_key]

class RMSProp:
    def __init__(self, lr, reg=0, momentum=0.9):
        self._lr = lr
        self._reg = reg
        self._cache = {}
        self._beta = beta
        self._eps = eps
    
    def _init_cache(self, layers: List[Layer]) -> None:
        for idx, l in enumerate(layers):
            if hasattr(l, 'W'):
                dw, db = l.W.grad, l.B.grad
                if dw is None or db is None:
                    continue

                dw_key, db_key = RMSProp._get_keys(idx)

                self._cache[dw_key] = np.zeros_like(dw)
                self._cache[db_key] = np.zeros_like(db)
    
    @staticmethod
    def _get_keys(idx):
        return f"dw{idx}", f"db{idx}"
    
    def update_weight(self, layers):
        if len(self._cache_s) == 0 or len(self._cache_v) == 0:
            self._init_cache(layers)

        if self._reg:
            for l in reversed(layers):
                if hasattr(l, 'W'):
                    l.W.grad += reg * 2 * l.W.value

        for idx, l in enumerate(layers):
            if hasattr(l, 'W'):
                dw_key, db_key = RMSProp._get_keys(idx)

                self._cache[dw_key] = self._beta * self._cache[dw_key] + \
                (1 - self._beta) * np.square(l.W.grad)
                self._cache[db_key] = self._beta * self._cache[db_key] + \
                    (1 - self._beta) * np.square(l.B.grad)

                dw = l.W.grad / (np.sqrt(self._cache[dw_key]) + self._eps)
                db = l.B.grad / (np.sqrt(self._cache[db_key]) + self._eps)

                l.W.value -= self._lr * dw
                l.B.value -= self._lr * db

# Criterion

In [49]:
class Criterion:
    def __init__(self, name, reduction=True):
        if name == 'CrossEntropy':
            self.__loss_func = self.__CrossEntropyLoss
        elif name == 'MSELoss':
            self.__loss_func = self.__MSELoss
        elif name == 'L1Loss':
            self.__loss_func = self.__L1Loss
        elif name == 'BCELoss':
            self.__loss_func = self.__BCELoss
        else:
            raise Exception("Invalid criteria!")
        if reduction:
            self.__reduc = np.mean
        else:
            self.__reduc = np.sum
    
    def __CrossEntropyLoss(self, pred_set, t_set):
        return -self.__reduc(np.log(pred_set[np.arange(len(t_set)), t_set.reshape(1, -1)]))
    
    def __MSELoss(self, pred_set, t_set):
        return self.__reduc((pred_set - t_set) ** 2)
    
    def __L1Loss(self, pred_set, t_set):
        return self.__reduc(np.linalg.norm(pred_set - t_set, axis=1))

    def __BCELoss(self, pred_set, t_set):
        return -self.__reduc(np.log(pred_set) * t_set + np.log(1 - pred_set) * (1 - t_set))
    
    def loss(self, pred_set, t_set):
        return self.__loss_func(pred_set, t_set)

# Metrics

In [50]:
def Accuracy(pred_set, t_set):
    return np.sum(pred_set.argmax(axis=1) == t_set) / t_set.shape[0]

def batchAccuracy(model, Loader):
    Acc, Num = 0, 0
    for batch in Loader:
        img = batch['imgs'] / 255
        targets = batch['targets']

        output = model.forward(img, False)
        Acc += np.sum(output.argmax(axis=1) == targets)
        Num += img.shape[0]
    return Acc / Num

def ConfusionMatrix(pred_set, t_set, norm=False):
    pred_labels = pred_set.argmax(axis=1)
    cm = np.zeros((pred_set.shape[1], pred_set.shape[1]))
    for i in range(len(t_set)):
        cm[int(t_set[i])][int(pred_labels[i])] += 1

# Training

Load CIFAR-10

In [51]:
!tar -xzf cifar-10-python.tar.gz

In [59]:
path_batches = 'cifar-10-batches-py/'
files = {'train': ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4'],
         'valid': ['data_batch_5'],
         'test': ['test_batch']}

TrainLoader = getLoader(path=path_batches, samples=files['train'], batch_size=25, shuff=True)
ValidLoader = getLoader(path=path_batches, samples=files['valid'], batch_size=25)
TestLoader = getLoader(path=path_batches, samples=files['test'], batch_size=25)

In [60]:
layers = [
    Conv2D(3, 32, kernel_size=(5, 5), stride=1, padding='same'),
    ReLU(),
    MaxPooling(pool_size=(2, 2), stride=2),
    Conv2D(32, 32, kernel_size=(5, 5), stride=1, padding='same'),
    ReLU(),
    MaxPooling(pool_size=(2, 2), stride=2),
    Conv2D(32, 64, kernel_size=(5, 5), stride=1, padding='same'),
    ReLU(),
    MaxPooling(pool_size=(2, 2), stride=2),
    Flattener(),
    FullyConnected(1024, 10),
    Softmax()
]

model = NeuralNetwork(layers)
model.set_optimizer(Adam(0.001))

In [61]:
from IPython import display

def train(model, TrainLoader, ValidLoader, epochs, criterion):
    train_loss = []
    val_loss = []
    for epoch in range(epochs):
        display.clear_output(wait=True)

        train_loss_batch = []
        val_loss_batch = []
        # Train
        for batch in TrainLoader:
            img = batch['imgs'] / 255
            targets = batch['targets']
            
            model.zero_grad()
            output = model.forward(img, training=True)
            model.backward(output, targets)
            model.step()

            output = model.forward(img, training=False)
            train_loss_batch.append(criterion.loss(output, targets))
        
        # Validation
        for batch in ValidLoader:
            img = batch['imgs'] / 255
            targets = batch['targets']
            
            output = model.forward(img, training=False)
            val_loss_batch.append(criterion.loss(output, targets))
        
        train_loss.append(np.nanmean(train_loss_batch))
        val_loss.append(np.nanmean(val_loss_batch))
        
        plt.figure('Training')
        plt.plot(train_loss, label='Train loss')
        plt.plot(val_loss, label='Valid loss')
        plt.legend()
        plt.show()

In [ ]:
criterion = Criterion('CrossEntropy')
train(model, TrainLoader, ValidLoader, 50, criterion)

In [ ]:
print('Test Accuracy - {}'.format(batchAccuracy(model, TestLoader)))

Test Accuracy - 0.575
